# Modules to be imported

In [55]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import overpy
import psycopg2
import statistics
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Webscraping from www.immoscout24.ch

In [92]:
# Create an empty array to store the scraped data
properties = []

# List of URLs to scrape

urls = [
    'https://www.immoscout24.ch/en/real-estate/rent/city-winterthur',
    'https://www.immoscout24.ch/en/real-estate/rent/city-zuerich',
    'https://www.immoscout24.ch/en/real-estate/rent/city-geneve',
    'https://www.immoscout24.ch/en/real-estate/rent/city-basel',
    'https://www.immoscout24.ch/en/real-estate/rent/city-lausanne',
    'https://www.immoscout24.ch/en/real-estate/rent/city-bern',
    'https://www.immoscout24.ch/en/real-estate/rent/city-luzern',
    'https://www.immoscout24.ch/en/real-estate/rent/city-st-gallen',
    'https://www.immoscout24.ch/en/real-estate/rent/city-lugano',
    'https://www.immoscout24.ch/en/real-estate/rent/city-biel-bienne'
]   


In [89]:
        
driver = webdriver.Chrome()

# Navigate to the search result page for ten selected cities on ImmoScout24.ch
driver.get('https://www.immoscout24.ch/en/real-estate/rent/city-zuerich')


# Wait for the page to load completely
#driver.implicitly_wait(10)

# Use BeautifulSoup to parse the HTML content of the page
soup = BeautifulSoup(driver.page_source, "html.parser")



#pages = soup.find("div",{"class": "Box-cYFBPY Flex-feqWzG FwNOn dCDRxm"}).find_all_next().text

#print(pages)

In [91]:
p = soup.find("div",{"class": "Box-cYFBPY Flex-feqWzG FwNOn dCDRxm"}).children
*_, last = p # for a better understanding check PEP 448

pages = last.text

print(pages)

25


In [93]:
# loop thorugh each URL 
def extract(link):
        L = []
        # Create a new instance of Chrome driver
        driver = webdriver.Chrome()

        # Navigate to the search result page for ten selected cities on ImmoScout24.ch
        driver.get(link)
        

        # Wait for the page to load completely
        driver.implicitly_wait(10)

        # Use BeautifulSoup to parse the HTML content of the page
        soup = BeautifulSoup(driver.page_source, "html.parser")

        #get last page number
        p = soup.find("div",{"class": "Box-cYFBPY Flex-feqWzG FwNOn dCDRxm"}).children
        *_, last = p # for a better understanding check PEP 448
        pages = last.text
        print("this is the amount of pages: ", pages)

        #iterate through all the pages
        for page in range(1,int(pages)+1):
            
            driver.get(link+'?pn='+str(page))

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Find all the property listings on the page
            listings = soup.find_all("a", {"class": "Wrapper__A-kVOWTT"})


        # Loop through the listings and extract the required information
            for listing in listings:
                #print(listing)
                # Get the price, size, and address of the property
                price = listing.find("h3", {"class": re.compile("^Box-cYFBPY hKJGPR Heading")}).text.strip()
                #size = listing.find("h3", {"class": re.compile("^Box-cYFBPY hKJGPR Heading")}).text.strip()
                address = listing.find("span", {"class": "AddressLine__TextStyled-eaUAMD"}).text.strip()

                # Add the property details to the properties array as a tuple
                L.append((price, address))
                #print(properties)
        
        # Close the Chrome driver
        driver.close()
        return L

In [94]:
import multiprocess as mp
import time
from multiprocessing import Process, Manager
from itertools import repeat
from functools import partial
import concurrent.futures

#timer for performance
start = time.perf_counter()

#trial and error with multiprocessing
#with mp.Pool() as pool:
#    result = pool.map(extract,urls)
#    print(result)

#Multithreading used to open multiple chrome windows
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor: 
    result = executor.map(extract,  urls)
    
finish = time.perf_counter()

print(f'Finished urls in {round(finish-start,2)} second(s)')

25
this is the amount of pages:  25
4
this is the amount of pages:  4
21
this is the amount of pages:  21
25
this is the amount of pages:  25
40
this is the amount of pages:  40
21
this is the amount of pages:  21
6
this is the amount of pages:  6
28
this is the amount of pages:  28
13
this is the amount of pages:  13
21
this is the amount of pages:  21
Finished urls in 3756.59 second(s)


In [95]:
r = list(result)

In [96]:
r

[[('4.5 rooms, 112 m², CHF 3970.—',
   'Ernst-Jung-Gasse 18, 8400 Winterthur, ZH'),
  ('3.5 rooms, 96 m², CHF 1755.—', 'Römerstr. 152, 8404 Winterthur, ZH'),
  ('3 rooms, 65 m², CHF 1480.—', 'Strahleggstrasse 68, 8400 Winterthur, ZH'),
  ('3.5 rooms, 83 m², CHF 1920.—', 'Hegistr. 35d, 8404 Winterthur, ZH'),
  ('2.5 rooms, 55 m², CHF 1770.—',
   'Wülflingerstrasse 155, 8408 Winterthur, ZH'),
  ('4.5 rooms, 105 m², CHF 1873.—',
   'Landvogt-Waser-Strasse 33, 8405 Winterthur, ZH'),
  ('1 room, 26 m², CHF 1195.—', 'Wartstr. 28, 8400 Winterthur, ZH'),
  ('4.5 rooms, 171 m², CHF 3339.—',
   'Stadthausstrasse 12, 8400 Winterthur, ZH'),
  ('6.5 rooms, 200 m², CHF 4450.—',
   'Schlosshofstrasse 218, 8404 Winterthur, ZH'),
  ('3.5 rooms, 103 m², CHF 2420.—', 'Eichgutstr. 4, 8400 Winterthur, ZH'),
  ('3 rooms, 70 m², CHF 2010.—', 'Schaffhauserstrasse 6, 8400 Winterthur, ZH'),
  ('1.5 rooms, 41 m², CHF 1130.—', 'Am Eulachpark 15, 8404 Winterthur, ZH'),
  ('2.5 rooms, 63 m², CHF 2335.—', 'Obertor 2

In [97]:
import csv
with open('immo.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(r)

In [137]:
import csv
immos = []
with open('immo.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(' '.join(row).split(','))
        immos.append(' '.join(row))

['"(\'4.5 rooms', ' 112 m²', " CHF 3970.—'", " 'Ernst-Jung-Gasse 18", ' 8400 Winterthur', ' ZH\')"', '"(\'3.5 rooms', ' 96 m²', " CHF 1755.—'", " 'Römerstr. 152", ' 8404 Winterthur', ' ZH\')"', '"(\'3 rooms', ' 65 m²', " CHF 1480.—'", " 'Strahleggstrasse 68", ' 8400 Winterthur', ' ZH\')"', '"(\'3.5 rooms', ' 83 m²', " CHF 1920.—'", " 'Hegistr. 35d", ' 8404 Winterthur', ' ZH\')"', '"(\'2.5 rooms', ' 55 m²', " CHF 1770.—'", " 'Wülflingerstrasse 155", ' 8408 Winterthur', ' ZH\')"', '"(\'4.5 rooms', ' 105 m²', " CHF 1873.—'", " 'Landvogt-Waser-Strasse 33", ' 8405 Winterthur', ' ZH\')"', '"(\'1 room', ' 26 m²', " CHF 1195.—'", " 'Wartstr. 28", ' 8400 Winterthur', ' ZH\')"', '"(\'4.5 rooms', ' 171 m²', " CHF 3339.—'", " 'Stadthausstrasse 12", ' 8400 Winterthur', ' ZH\')"', '"(\'6.5 rooms', ' 200 m²', " CHF 4450.—'", " 'Schlosshofstrasse 218", ' 8404 Winterthur', ' ZH\')"', '"(\'3.5 rooms', ' 103 m²', " CHF 2420.—'", " 'Eichgutstr. 4", ' 8400 Winterthur', ' ZH\')"', '"(\'3 rooms', ' 70 m²', "

In [338]:
df = pd.DataFrame()

In [377]:
immo0 = immos[9].replace('"','').replace('\'','')

In [378]:
immo0

'(3.5 rooms, 100 m², CHF 1990.—, Feldeckstrasse 15, 2502 Biel/Bienne, BE),(4.5 rooms, 116 m², CHF 2350.—, Rüschlistrasse 8, 2502 Biel/Bienne, BE),(2.5 rooms, 58 m², CHF 1140.—, Friedweg 8, 2503 Biel/Bienne, BE),(3.5 rooms, 75 m², CHF 1570.—, Friedweg 8, 2503 Biel/Bienne, BE),(3.5 rooms, 67 m², CHF 1195.—, Südstrasse 36, 2504 Biel/Bienne, BE),(2.5 rooms, 80 m², CHF 1600.—, Freiestrasse 52, 2502 Biel/Bienne, BE),(80 m², CHF 1600.—, Freiestrasse 52, 2502 Biel/Bienne, BE),(4.5 rooms, 115 m², CHF 2045.—, Salomegasse 14, 2500 Biel/Bienne, BE),(4.5 rooms, 144 m², CHF 2240.—, Alleestrasse 37, 2503 Biel/Bienne, BE),(2.5 rooms, 74 m², CHF 1455.—, Neumarktstrasse 29, 2500 Biel/Bienne, BE),(4.5 rooms, 140 m², CHF 2225.—, Neumarktstrasse 31, 2500 Biel/Bienne, BE),(2 rooms, 30 m², CHF 960.—, Bahnhofstrasse 9, 2502 Biel/Bienne, BE),(3.5 rooms, 73 m², CHF 1390.—, Friedweg 8, 2503 Biel/Bienne, BE),(2.5 rooms, 65 m², CHF 1300.—, Friedweg 8, 2503 Biel/Bienne, BE),(3.5 rooms, 75 m², CHF 1725.—, Parkweg 10

In [366]:
immo1 = re.split(r'[()]', immo0)

In [367]:
immo2 = [x for x in immo1 if not(x=='' or x==',')]

In [368]:
df = pd.DataFrame(immo2)

In [369]:
df = df[0].str.split(',', expand=True)

In [370]:
pd.set_option('display.max_rows', None)

In [371]:
df2 = df[df[1].str.contains('m²')==True]

In [372]:
len(df2)

350

In [373]:
len(df)

493

In [376]:
df2

,0,1,2,3,4,5,6
2,4 rooms,83 m²,CHF 1940.—,Robinsonweg 44,3006 Bern,BE,None
3,4 rooms,98 m²,CHF 2600.—,Sandrainstrasse 91,3007 Bern,BE,None
4,3.5 rooms,75 m²,CHF 2910.—,Zeughausgasse 27,Wohnungen,3011 Bern,BE
5,2.5 rooms,105 m²,CHF 1860.—,Sickingerstrasse 3,3014 Bern,BE,None
6,4.5 rooms,110 m²,CHF 2050.—,Brünnenstrasse 63,3018 Bern,BE,None
7,3 rooms,74 m²,CHF 2500.—,Marzilistrasse 38,3005 Bern,BE,None
8,1.5 rooms,42 m²,CHF 1480.—,Muristrasse 64,3006 Bern,BE,None
9,1.5 rooms,40 m²,CHF 1440.—,Muristrasse 64,3006 Bern,BE,None
11,5.5 rooms,146 m²,CHF 6100.—,3000 Bern,BE,None,None
12,3 rooms,115 m²,CHF 2650.—,Münstergasse 48,3011 Bern,BE,None


# WEB API using Overpass Turbo

In [7]:
# defining the overpass query
query = '''
[out:json];
{{geocodeArea:Zurich}} -> .searchArea0;
{{geocodeArea:Geneva}} -> .searchArea1;
{{geocodeArea:Basel}} -> .searchArea2;
{{geocodeArea:Lausanne}} -> .searchArea3;
{{geocodeArea:Bern}} -> .searchArea4;
{{geocodeArea:Winterthur}} -> .searchArea5;
{{geocodeArea:Luzern}} -> .searchArea6;
{{geocodeArea:St. Gallen}} -> .searchArea7;
{{geocodeArea:Lugano}} -> .searchArea8;
{{geocodeArea:Biel}} -> .searchArea9;
( area.searchArea0; area.searchArea1; area.searchArea2; area.searchArea3; area.searchArea4; area.searchArea5; area.searchArea6; area.searchArea7; area.searchArea8; area.searchArea9;) -> .searchArea;
(node["shop"="supermarket"](area.searchArea);way["shop"="supermarket"](area.searchArea););
out center;
'''

# creating Overpass API object
api = overpy.Overpass()

# performing the query and retrieveing the results
response = api.query(query)

# print the name and zip code of each supermarket
for element in response["elements"]:
    if element["tags"].get("name") and element["tags"].get("addr:postcode"):
        name = element["tags"]["name"]
        zip_code = element["tags"]["addr:postcode"]
        data = (name, zip_code)

OverpassBadRequest: Error: line 3: parse error: Unknown type &quot;{&quot; 
Error: line 3: parse error: An empty query is not allowed 
Error: line 3: parse error: ';' expected - '{' found. 
Error: line 4: parse error: Unknown type &quot;{&quot; 
Error: line 4: parse error: An empty query is not allowed 
Error: line 4: parse error: ';' expected - '{' found. 
Error: line 5: parse error: Unknown type &quot;{&quot; 
Error: line 5: parse error: An empty query is not allowed 
Error: line 5: parse error: ';' expected - '{' found. 
Error: line 6: parse error: Unknown type &quot;{&quot; 
Error: line 6: parse error: An empty query is not allowed 
Error: line 6: parse error: ';' expected - '{' found. 
Error: line 7: parse error: Unknown type &quot;{&quot; 
Error: line 7: parse error: An empty query is not allowed 
Error: line 7: parse error: ';' expected - '{' found. 
Error: line 8: parse error: Unknown type &quot;{&quot; 
Error: line 8: parse error: An empty query is not allowed 
Error: line 8: 

# Connection to AWS PostgreSQL and inserting Data

In [ ]:
#Connection to the database
conn = psycopg2.connect(
    host="ads-database1.cbwqb3cep5ch.eu-central-1.rds.amazonaws.com",
    database="adsdatabase",
    user="adschief1",
    password="1eYvmDnMPikKSImeLrev"
)

# Creating a cursor object for executing SQL statements
cur = conn.cursor()

# Iterate over the properties array
for property in properties:
    # Extract the property details
    details = property[0].split(", ")  # Split the details by comma and space

    # Extract the individual details
    rooms = details[0]
    size = details[1]
    price = details[2]
    address = details[3]
    zipcode = details[4]
    kanton = details [5]
    
    # Process the individual data record as needed
    print("Rooms:", rooms)
    print("Size:", size)
    print("Price:", price)
    print("Address:", address)
    print("Zip-Code", zipcode)
    print("Kanton", kanton)
    print("--------------------")


    # Insert the data from ImmoScout24 into the database
    cur.execute(
        "INSERT INTO your_table_name (rooms, size, price, address, zipcode, kanton) VALUES (%s, %s, %s, %s. %s, %s)",
        (rooms, size, price, address, zipcode, kanton)
    )
    
    # Insert the data from Overpass Turbo into the database
    cur.execute(
        "INSERT INTO your_table_name2 (name, zip_code) VALUES (%s, %s)"
        (name, zip_code)
    )

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


#### Join the two Tables "immoscout24_data" and "overpass_data"

In [ ]:
# Join the two tables 
query = '''
SELECT *
FROM immoscout24_data
JOIN overpass_data ON immoscout24_data.zip_code = overpass_data.zip_code
'''
data_df = pd.read_sql(query, conn)


# EDA using our data from PostgreSQL

In [ ]:
# First few rows of the DataFrame
data_df.head()

In [ ]:
# Summary statistics
data_df.describe()

In [ ]:
# Examine the data types
data_df.dtypes

In [ ]:
# Count number of missing values
data_df.isnull().sum()

In [ ]:
#mean, median, mean, median, mode, variance, standard deviation
#histogram, boxplots, scatter plots
#correlation analysis
#hypothesis testing 

# Retrieve the data from the database
cur.execute("SELECT price FROM XXXXXXXXXXXX")

# Fetch all the rows of the query result
rows = cur.fetchall()

# Extract the values from the rows
prices = [row[0] for row in rows]

# Calculate the mean
mean = statistics.mean(prices)

# Calculate the median
median = statistics.median(prices)

# Calculate the mode
mode = statistics.mode(prices)

# Calculate the variance
variance = statistics.variance(prices)

# Calculate the standard deviation
std_dev = statistics.stdev(prices)

# Print the results
print("Mean:", mean)
print("Median:", median)
print("Mode:", mode)
print("Variance:", variance)
print("Standard Deviation:", std_dev)

# Correlation Matrix 

### How do we separate the data into training and testing data? NO CLUE

In [ ]:
# Split the data into training and test sets (70% training, 30% testing)
train_df, test_df = train_test_split(data_df, test_size=0.3, random_state=42)


# Create the correlation matrix
correlation_matrix = train_df.corr()

print(correlation_matrix)

### Correlation Matrix Plot

In [ ]:
# Correlation Matrix Plot (corrplot)
sns.pairplot(train_df)